**Table of Contents:**
- Introduction
- Architecture Overview
- ConvNet Layers
    - Convolutional Layer
    - Pooling Layer
    - Normalization Layer
    - Fully-Connected Layer
    - Converting Fully-Connected Layers to Convolutional Layers
- ConvNet Architectures
    - Layer Patterns
    - Layer Sizing Patterns
    - Case Studies (LeNet / AlexNet / ZFNet / GoogLeNet / VGGNet)
    - Computational Considerations
- Additional References

## Introduction: Convolutional Neural Networks (CNNs/ConvNets)

`Convolutional Neural Networks` are very similar to ordinary Neural Networks from the previous chapter: 
- They are made up of neurons that have learnable `weights` and `biases`. 
- Each neuron receives some inputs, performs a dot product and optionally follows it with a non-linearity. 
- The whole network still expresses a single differentiable score function: from the raw image pixels on one end to class scores at the other. 
-They have a loss function (e.g. SVM/Softmax) on the last (`fully-connected`) layer and all the tips/tricks we developed for learning regular Neural Networks still apply.

So what changes? ConvNet architectures make the explicit assumption that the **inputs are images**, which allows us to encode certain properties into the architecture. These then make the `forward function` more efficient to implement and vastly reduce the amount of parameters in the network.

## Architecture Overview

**Recall: Regular Neural Nets**. 
- Neural Networks receive an input (a single vector), and transform it through a series of hidden layers. 
- Each hidden layer is made up of a set of neurons, where each neuron is *fully connected* to all neurons in the previous layer, and where neurons in a single layer function completely independently and do not share any connections. 
- The last `fully-connected` layer is called the “output layer” and in classification settings it represents the class scores.

**Regular Neural Nets don’t scale well to full images**. In CIFAR-10, images are only of size 32x32x3 (32 wide, 32 high, 3 color channels), so a single fully-connected neuron in a first hidden layer of a regular Neural Network would have 32*32*3 = 3072 weights. This amount still seems manageable, but clearly this fully-connected structure does not scale to larger images. For example, an image of more respectable size, e.g. 200x200x3, would lead to neurons that have 200*200*3 = 120,000 weights. Moreover, we would almost certainly want to have several such neurons, so the parameters would add up quickly! Clearly, **this full connectivity is wasteful and the huge number of parameters would quickly lead to overfitting**.

**3D volumes of neurons**. Convolutional Neural Networks take advantage of the fact that the input consists of images and they constrain the architecture in a more sensible way. In particular, unlike a regular Neural Network, the layers of a ConvNet have neurons arranged in 3 dimensions: width, height, depth. (Note that the word depth here refers to the third dimension of an activation volume, not to the depth of a full Neural Network, which can refer to the total number of layers in a network.) 
<br>For example, the input images in CIFAR-10 are an input volume of activations, and the volume has dimensions 32x32x3 (width, height, depth respectively). As we will soon see, the neurons in a layer will only be **connected to a small region of the layer before it**, instead of all of the neurons in a fully-connected manner. 
<br>Moreover, the final output layer would for CIFAR-10 have dimensions 1x1x10, because by the end of the ConvNet architecture we will reduce the full image into a single vector of class scores, arranged along the depth dimension. 

Here is a visualization:

A regular 3-layer Neural Network|A ConvNet arranges its neurons in three dimensions (width, height, depth), as visualized in one of the layers
:-:|:-:
![](./Piks/neural_net2.jpeg)|![](./Piks/cnn.jpeg)

*Every layer of a ConvNet transforms the 3D input volume to a 3D output volume of neuron activations. In this example, the red input layer holds the image, so its width and height would be the dimensions of the image, and the depth would be 3 (Red, Green, Blue channels).*

> **A ConvNet is made up of Layers. Every Layer has a simple API: It transforms an input 3D volume to an output 3D volume with some differentiable function that may or may not have parameters.**

## Layers used to build ConvNets

As we described above, a simple ConvNet is a sequence of layers, and every layer of a ConvNet transforms one volume of activations to another through a differentiable function. 
<br>We use **three** main types of layers to build ConvNet architectures: 
- **Convolutional Layer**.
- **Pooling Layer**.
- **Fully-Connected Layer** (exactly as seen in regular Neural Networks). 

We will stack these layers to form a full **ConvNet architecture**.

**Example Architecture: Overview**. We will go into more details below, but a simple ConvNet for CIFAR-10 classification could have the architecture **\[INPUT - CONV - RELU - POOL - FC\]**. In more detail:
- **INPUT** \[32x32x3\] will hold the **raw pixel values** of the image, in this case an image of width 32, height 32, and with three color channels R,G,B.
- **CONV layer** will **compute the output of neurons that are connected to local regions in the input**, each computing a dot product between their weights and a small region they are connected to in the input volume. This may result in volume such as \[32x32x12\] if we decided to use 12 filters.
- **RELU layer** will apply an **elementwise activation function**, such as the max(0,x) thresholding at zero. This leaves the size of the volume unchanged (\[32x32x12\]).
- **POOL layer** will perform a **downsampling** operation along the spatial dimensions (width, height), resulting in volume such as \[16x16x12\].
- **FC (fully-connected) layer** will compute the class scores, resulting in volume of size \[1x1x10\], where each of the 10 numbers correspond to a class score, such as among the 10 categories of CIFAR-10. As with ordinary Neural Networks and as the name implies, each neuron in this layer will be connected to all the numbers in the previous volume.

In this way, ConvNets transform the original image layer by layer from the original pixel values to the final class scores. Note that **some layers contain parameters and other don’t**. 
<br>In particular, the CONV/FC layers perform transformations that are a function of not only the **activations** in the input volume, but also of the **parameters** (the weights and biases of the neurons). 
<br>On the other hand, the RELU/POOL layers will implement a **fixed function**. 
<br>The parameters in the CONV/FC layers will be trained with gradient descent so that the class scores that the ConvNet computes are consistent with the labels in the training set for each image.

In summary:
- A ConvNet architecture is in the simplest case a list of Layers that transform the image volume into an output volume (e.g. holding the class scores)
- There are a few distinct types of Layers (e.g. CONV/FC/RELU/POOL are by far the most popular)
- Each Layer accepts an input 3D volume and transforms it to an output 3D volume through a differentiable function
- Each Layer may or may not have parameters (e.g. CONV/FC do, RELU/POOL don’t)
- Each Layer may or may not have additional hyperparameters (e.g. CONV/FC/POOL do, RELU doesn’t)

![](./Piks/convnet.jpeg)

*The activations of an example ConvNet architecture. The initial volume stores the raw image pixels (left) and the last volume stores the class scores (right). Each volume of activations along the processing path is shown as a column. Since it's difficult to visualize 3D volumes, we lay out each volume's slices in rows. The last layer volume holds the scores for each class, but here we only visualize the sorted top 5 scores, and print the labels of each one. The full web-based demo is shown in the header of our website. The architecture shown here is a tiny VGG Net, which we will discuss later.*

### Convolutional Layer

The Conv layer is the core building block of a Convolutional Network that does most of the computational heavy lifting.

**Overview and intuition without brain stuff.**
<br>Let’s first discuss what the CONV layer computes without brain/neuron analogies. 
- The CONV layer's parameters consist of a set of learnable filters. 
- Every filter is small spatially (along width and height), but extends through the full depth of the input volume. 

For example: 
- A typical filter on a first layer of a ConvNet might have size 5x5x3 (i.e. 5 pixels width and height, and 3 because images have depth 3, the color channels). 
- During the forward pass, we slide (more precisely, convolve) each filter across the width and height of the input volume and compute dot products between the entries of the filter and the input at any position. 
- As we slide the filter over the width and height of the input volume, we will produce a 2-dimensional activation map that gives the responses of that filter at every spatial position. 
- Intuitively, the network will learn filters that activate when they see some type of visual feature such as an edge of some orientation or a blotch of some color on the first layer, or eventually entire honeycomb or wheel-like patterns on higher layers of the network. 
- Now, we will have an entire set of filters in each CONV layer (e.g. 12 filters), and each of them will produce a separate 2-dimensional activation map. We will stack these activation maps along the depth dimension and produce the output volume.

**The brain view.** 
<br>If you’re a fan of the brain/neuron analogies, every entry in the 3D output volume can also be interpreted as an output of a neuron that looks at only a small region in the input and shares parameters with all neurons to the left and right spatially (since these numbers all result from applying the same filter). We now discuss the details of the neuron connectivities, their arrangement in space, and their parameter sharing scheme.

**Local Connectivity.**
<br>When dealing with high-dimensional inputs such as images, as we saw above it is impractical to connect neurons to all neurons in the previous volume. 
<br>Instead, we will connect each neuron to only **a local region** of the input volume. The spatial extent of this connectivity is a hyperparameter called the **receptive field of the neuron** (equivalently this is the filter size). The extent of the connectivity along the depth axis is always equal to the depth of the input volume. It is important to emphasize again this asymmetry in how we treat the spatial dimensions (width and height) and the depth dimension: The connections are local in space (along width and height), but always full along the entire depth of the input volume.

continue later